In [1]:
import numpy as np
np.random.seed(10)
import random
random.seed(10)

#mv=np.array([[0.4,0.5,0.5],[0.5,0.4,0.5],[0.5,0.5,0.4]]).transpose()
# mv=np.array([[0.47,0.51,0.5],[0.47,0.5,0.51],[0.5,0.51,0.49]])
#mv=np.array([[0.4,0.50,0.7],[0.4,0.5,0.6],[0.3,0.5,0.7]]) $this worked
#mv=np.array([[0.45,0.55,0.7],[0.45,0.55,0.7]]) #this worked*
#mv=np.array([[0.40,0.49,0.52,0.53],[0.52,0.51,0.53,0.54]]) #this worked* best so far
#mv=np.array([[0.35,0.55,0.65],[0.35,0.6,0.70]]) #this worked
mv=np.array([[0.39,0.49,0.51,0.53],[0.52,0.51,0.53,0.54]]) #this worked* 

std=0.2

In [4]:
!pip install numpy

In [3]:
As=[]
items=4
agents=2
for idx in range(items):
    for jdx in range(items):
        if idx!=jdx:
            A = np.zeros((agents,items))
            A[0][idx]=1
            A[1][jdx]=1
            As.append(A)

In [7]:
As

[array([[1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 array([[1., 0., 0., 0.],
        [0., 0., 1., 0.]]),
 array([[1., 0., 0., 0.],
        [0., 0., 0., 1.]]),
 array([[0., 1., 0., 0.],
        [1., 0., 0., 0.]]),
 array([[0., 1., 0., 0.],
        [0., 0., 1., 0.]]),
 array([[0., 1., 0., 0.],
        [0., 0., 0., 1.]]),
 array([[0., 0., 1., 0.],
        [1., 0., 0., 0.]]),
 array([[0., 0., 1., 0.],
        [0., 1., 0., 0.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[0., 0., 0., 1.],
        [1., 0., 0., 0.]]),
 array([[0., 0., 0., 1.],
        [0., 1., 0., 0.]]),
 array([[0., 0., 0., 1.],
        [0., 0., 1., 0.]])]

In [55]:
from scipy.stats import skewnorm
import scipy.stats as stats
from scipy.stats import skewnorm

import numpy as np
from   scipy.stats import (multivariate_normal as mvn,
                           norm)
from   scipy.stats._multivariate import _squeeze_output

skew = -10

def filter_valuations(points, mean, std_dev, alpha=0.04):
    mean = np.array(mean)
    print("mean shape", mean.shape)
    print("std shape", std_dev.shape)
    delta = alpha/np.prod(points[0].shape)
    print(alpha)
    print(delta)
    print(np.prod(points[0].shape))
    alpha = delta

    std_dev_t = std_dev.transpose()
    print("std dev transpose",std_dev_t)
    
    
    # Calculate the z-critical value based on the confidence level
    print("delta",delta)
    print("alpha",alpha)
    mean_transpose = mean.transpose()

    lower_bound1 = skewnorm.ppf(alpha / 2, skew, loc=mean_transpose[0,:], scale=0.01)
    lower_bound2 = skewnorm.ppf(alpha / 2, skew, loc=mean_transpose[1,:], scale=0.04)
    lower_bound3 = skewnorm.ppf(alpha / 2, skew, loc=mean_transpose[2,:], scale=0.05)
    lower_bound4 = skewnorm.ppf(alpha / 2, skew, loc=mean_transpose[3,:], scale=0.09)


    upper_bound1 = skewnorm.ppf(1 - alpha / 2, skew, loc=mean_transpose[0,:], scale=0.01)
    upper_bound2 = skewnorm.ppf(1 - alpha / 2, skew, loc=mean_transpose[1,:], scale=0.04)

    upper_bound3 = skewnorm.ppf(1 - alpha / 2, skew, loc=mean_transpose[2,:], scale=0.05)

    upper_bound4 = skewnorm.ppf(1 - alpha / 2, skew, loc=mean_transpose[3,:], scale=0.09)

    lower_bound = np.vstack((lower_bound1, lower_bound2, lower_bound3, lower_bound4)).transpose() 
    upper_bound = np.vstack((upper_bound1, upper_bound2, upper_bound3, upper_bound4)).transpose() 

    print("lower bound",lower_bound)
    print("upper bound",upper_bound)



    new_valuations=[]
    for point in points:
        point = np.array(point)
    
        # Check if the random variable lies within the confidence interval
        flag = np.logical_and(np.less_equal(point, upper_bound),np.greater_equal(point,lower_bound))
        #print(flag)
        is_valid = np.all(flag==1)
        if is_valid==1:
            point = np.clip(point,0,1)
            new_valuations.append(point)

    print("selected points", len(new_valuations))
    
            
    
    return new_valuations



class multivariate_skewnorm:
    
    def __init__(self, shape, cov=None):
        self.dim   = len(shape)
        self.shape = np.asarray(shape)
        self.mean  = np.zeros(self.dim)
        self.cov   = np.eye(self.dim) if cov is None else np.asarray(cov)

    def pdf(self, x):
        return np.exp(self.logpdf(x))
        
    def logpdf(self, x):
        x    = mvn._process_quantiles(x, self.dim)
        pdf  = mvn(self.mean, self.cov).logpdf(x)
        cdf  = norm(0, 1).logcdf(np.dot(x, self.shape))
        return _squeeze_output(np.log(2) + pdf + cdf)


    def rvs_fast(self, size=1):
        aCa      = self.shape @ self.cov @ self.shape
        delta    = (1 / np.sqrt(1 + aCa)) * self.cov @ self.shape
        cov_star = np.block([[np.ones(1),     delta],
                             [delta[:, None], self.cov]])
        x        = mvn(np.zeros(self.dim+1), cov_star).rvs(size)
        x0, x1   = x[:, 0], x[:, 1:]
        inds     = x0 <= 0
        x1[inds] = -1 * x1[inds]
        return x1



def get_noise_normal(num=40000):
    print(mv.shape)
    k = mv.shape[0]
    mv_skew1 = multivariate_skewnorm(shape=np.ones(k)*-4,cov=np.eye(k)*0.01)
    mv_skew2 = multivariate_skewnorm(shape=np.ones(k)*-4, cov=np.eye(k)*0.02) #0.02 #0.09

    mv_skew3 = multivariate_skewnorm(shape=np.ones(k)*-4, cov=np.eye(k)*0.05)
    mv_skew4 = multivariate_skewnorm(shape=np.ones(k)*-4, cov=np.eye(k)*0.09)

    # mv_skew1 = skewnorm.rvs(-4, loc=0, scale=0.01, size=np.ones(k))
    # mv_skew2 = skewnorm.rvs(-4, loc=0, scale=0.02, size=np.ones(k))
    # mv_skew3 = skewnorm.rvs(-4, loc=0, scale=0.05, size=np.ones(k))
    # mv_skew4 = skewnorm.rvs(-4, loc=0, scale=0.09, size=np.ones(k))
    
    



    #best
    # noise1 = skewnorm.rvs(-5,scale=0.01,size=(num,mv.shape[0]))#mv_skew1.rvs_fast(num)
    # noise2 = skewnorm.rvs(-5,scale=0.04,size=(num,mv.shape[0])) #mv_skew2.rvs_fast(num)
    # noise3 = skewnorm.rvs(-5,scale=0.05,size=(num,mv.shape[0])) #mv_skew3.rvs_fast(num)
    # noise4 = skewnorm.rvs(-5,scale=0.09,size=(num,mv.shape[0])) #mv_skew4.rvs_fast(num)

    noise1 = skewnorm.rvs(-5,scale=0.01,size=(num,mv.shape[0]))#mv_skew1.rvs_fast(num)
    noise2 = skewnorm.rvs(-5,scale=0.04,size=(num,mv.shape[0])) #mv_skew2.rvs_fast(num)
    noise3 = skewnorm.rvs(-5,scale=0.05,size=(num,mv.shape[0])) #mv_skew3.rvs_fast(num)
    noise4 = skewnorm.rvs(-5,scale=0.09,size=(num,mv.shape[0])) #mv_skew4.rvs_fast(num)

    # noise1 = skewnorm.rvs(-4,scale=0.01,size=(num,mv.shape[0]))#mv_skew1.rvs_fast(num)
    # noise2 = skewnorm.rvs(-4,scale=0.04,size=(num,mv.shape[0])) #mv_skew2.rvs_fast(num)
    # noise3 = skewnorm.rvs(-4,scale=0.07,size=(num,mv.shape[0])) #mv_skew3.rvs_fast(num)
    # noise4 = skewnorm.rvs(-4,scale=0.09,size=(num,mv.shape[0])) #mv_skew4.rvs_fast(num)

    

    noise = np.vstack((noise1,noise2,noise3, noise4))
    print("noise shape",noise.shape)
    noise = noise.reshape((4,noise1.shape[0],mv.shape[0]))
    print("shape", noise.shape)
    
    noise = noise.transpose((1,0,2))
    print("after tr", noise.shape)
    noise = noise.transpose((0,2,1))
    print("after tr", noise.shape)
    


    valuations = np.repeat(mv.reshape((-1,mv.shape[0],mv.shape[1])),num,axis=0) + noise

    print("sum",np.sum(valuations<0) + np.sum(valuations>1))
    
    
    # valuations = np.clip(valuations,0,1)
    # valuations = valuations.reshape((-1,8))
    print(valuations.shape)
    



    # recompute mean and variance after clipping
    variance = np.var(valuations, axis=0)
    print("var",variance)
    print("std dev", np.sqrt(variance))
    mean_val = np.mean(valuations,axis=0)
    print("shape",valuations.shape)
    valuations_ = np.copy(valuations)
    shape = valuations_.shape
    valuations_ = valuations.reshape((shape[0],shape[1]*shape[2]))
    print("cov1 shape",valuations_.shape)
    var = np.var(valuations,axis=0)
    # print("cov shape",cov.shape)
    print("mean val",mean_val)
    print("mv",mv)
    true_mean = mv
    std1 = np.ones(mv.shape[0])*0.01
    std2 = np.ones(mv.shape[0])*0.04
    std3 = np.ones(mv.shape[0])*0.05
    std4 = np.ones(mv.shape[0])*0.09
    std = np.vstack((std1,std2,std3,std4))
    


    true_std = std.transpose()


    
    
    return valuations, mean_val, var, true_mean, true_std

valuations, mean_val, var, true_mean, true_std = get_noise_normal()
valuations1, mean_val1, var1, true_mean, true_std = get_noise_normal()





print(valuations.shape)
print(np.repeat(mv.reshape((-1,mv.shape[0],mv.shape[1])),1000,axis=0).shape)


(2, 4)
noise shape (160000, 2)
shape (4, 40000, 2)
after tr (40000, 4, 2)
after tr (40000, 2, 4)
sum 0
(40000, 2, 4)
var [[3.83528713e-05 6.13082572e-04 9.71646644e-04 3.17891400e-03]
 [3.90234050e-05 6.18330604e-04 9.71367237e-04 3.14548547e-03]]
std dev [[0.00619297 0.0247605  0.03117125 0.05638186]
 [0.00624687 0.02486625 0.03116676 0.05608463]]
shape (40000, 2, 4)
cov1 shape (40000, 8)
mean val [[0.38222114 0.45885851 0.47079937 0.45940167]
 [0.51220144 0.47858452 0.49095673 0.46948244]]
mv [[0.39 0.49 0.51 0.53]
 [0.52 0.51 0.53 0.54]]
(2, 4)
noise shape (160000, 2)
shape (4, 40000, 2)
after tr (40000, 4, 2)
after tr (40000, 2, 4)
sum 0
(40000, 2, 4)
var [[3.86526800e-05 6.25385006e-04 9.80669408e-04 3.15286917e-03]
 [3.86590944e-05 6.25450533e-04 9.87537393e-04 3.14634414e-03]]
std dev [[0.00621713 0.0250077  0.03131564 0.05615042]
 [0.00621764 0.02500901 0.03142511 0.05609228]]
shape (40000, 2, 4)
cov1 shape (40000, 8)
mean val [[0.38218999 0.45870757 0.47068699 0.45941637]
 [0.

In [57]:
print(var.shape)

(2, 4)


In [59]:
valuations[2]

array([[0.38772597, 0.41790562, 0.49536754, 0.42849402],
       [0.51330045, 0.44888681, 0.49584227, 0.52891599]])

In [61]:
def compute_optimal_allocation_single(As,valuations, egal=False):
    idx=0
    results={}
    n = valuations.shape[0]
    for idx in range(n):
        valuation = valuations[idx]
        
        res=[]
        for alloc in As:
            #print(alloc.flatten())
            if egal==False:
                val = np.sum(valuation.flatten()*alloc.flatten())
            else:
                val = np.min(np.sum(valuation*alloc,axis=1))
            res.append(val)
        res = np.array(res)*-1
        results[idx]=(np.sort(res),np.argsort(res))
        
    return results
    
            
            
        
        


def compute_optimal_allocation_pair(As,valuations):
    idx=0
    results={}
    n = len(valuations)
    for idx in range(n):
        for jdx in range(idx+1,n):
    
            res=[]
            for alloc in As:
                val1 = np.sum(valuations[idx].flatten()*alloc.flatten())
                val2 = np.sum(valuations[jdx].flatten()*alloc.flatten())
                res.append((0.2*val1 + 0.20*val2)/(0.4))
            
                
            res = np.array(res)*-1
            results[(idx,jdx)]=(np.sort(res),np.argsort(res))
    return results
    

In [63]:
print(mv)

[[0.39 0.49 0.51 0.53]
 [0.52 0.51 0.53 0.54]]


In [65]:
res_avg=compute_optimal_allocation_single(As,np.array([mv]))
print(res_avg)

{0: (array([-1.06, -1.05, -1.05, -1.04, -1.03, -1.03, -1.02, -1.02, -1.01,
       -0.93, -0.92, -0.9 ]), array([11,  8,  9, 10,  5,  6,  4,  7,  3,  2,  1,  0], dtype=int64))}


In [67]:
res_avg1=compute_optimal_allocation_single(As,np.array([mean_val]))
print(res_avg1)

{0: (array([-0.98300081, -0.97160311, -0.97105996, -0.9503584 , -0.94981524,
       -0.94938389, -0.94028181, -0.93798619, -0.92834095, -0.87317787,
       -0.86080566, -0.85170358]), array([ 6,  9,  3, 11,  4,  7,  8, 10,  5,  1,  0,  2], dtype=int64))}


In [69]:
#As=[A1,A2,A3,A4,A5,A6]

# results=compute_optimal_allocation_single(As,valuations)
# for idx in range(100):
#     print(f"idx {idx} res {results[idx]}")

In [71]:
#pair_results = compute_optimal_allocation_pair(As,valuations)

In [73]:
def compute_worst(As, valuations):
    all_vals=[]
    for idx in range(6):
        res=[]
        n = len(valuations)
        for jdx in range(n):
            val = np.sum(valuations[jdx].flatten()*As[idx].flatten())
            res.append(val)
        all_vals.append(np.min(res))
    return np.max(all_vals), np.argmax(all_vals)


        
        

def compute_cvar(As,valuations,p=0.01):
    all_vals=[]
    for idx in range(len(As)):
        res=[]
        n = len(valuations)
        for jdx in range(n):
            val = np.sum(valuations[jdx].flatten()*As[idx].flatten())
            res.append(val)
        all_vals.append(np.mean(np.sort(res[:int(p*n)])))
    return np.max(all_vals), np.argmax(all_vals)


    

In [75]:
def compute_egal_worst(As, valuations):
    all_vals=[]
    for idx in range(6):
        res=[]
        n = len(valuations)
        for jdx in range(n):
            val = np.min(np.sum(valuations[jdx]*As[idx],axis=1))
            res.append(val)
        all_vals.append(np.min(res))
    return np.max(all_vals), np.argmax(all_vals)


        
        

def compute_egal_cvar(As,valuations,p):
    all_vals=[]
    for idx in range(len(As)):
        res=[]
        n = len(valuations)
        for jdx in range(n):
            val = np.min(np.sum(valuations[jdx]*As[idx],axis=1))
            res.append(val)
        
        all_vals.append(np.mean(np.sort(res[:int(p*n)])))
    print(f"{np.max(all_vals)}")
    return np.max(all_vals), np.argmax(all_vals)


    

In [77]:

def train(p):
    avg=5
    print(valuations.shape)
    print(mean_val.shape)
    # print(cov.shape)
    # print(p.shape)
    new_valuations = filter_valuations(valuations, true_mean, true_std,p)
    new_valuations1 = filter_valuations(valuations1, true_mean, true_std,p)

    w_res=compute_worst(As,new_valuations)
    c_res=compute_cvar(As,valuations,p)
    print("worst opt",compute_worst(As,new_valuations))
    print("cvar opt",compute_cvar(As,valuations,p))
    print("**********")
    print("P",p)
    res_avg=compute_optimal_allocation_single(As,np.array([mv]),egal=False)
    #print(res_avg[0][1])
    
    print("avg assigment",As[res_avg[0][1][0]])
    print("worst assignment",As[w_res[1]])
    print("cvar assignment",As[c_res[1]])
    print("Train Avg perf", compute_cvar([As[res_avg[0][1][0]]],valuations,p))
    print("Train Worst perf", compute_cvar([As[w_res[1]]],valuations,p))
    print("Train CVaR perf", compute_cvar([As[c_res[1]]],valuations,p))
    print("##################")

    print("Test Avg perf", compute_cvar([As[res_avg[0][1][0]]],valuations1,p))

    print("Test Worst perf", compute_cvar([As[w_res[1]]],valuations1,p))
    print("Test CVaR perf", compute_cvar([As[c_res[1]]],valuations1,p))
    print("**********")
    
        

In [79]:

def train_egal(p):
    avg=5
    print(valuations.shape)
    print(mean_val.shape)
    # print(cov.shape)
    # print(p.shape)
    new_valuations = filter_valuations(valuations, true_mean, true_std,p)
    new_valuations1 = filter_valuations(valuations1, true_mean, true_std,p)


    w_res=compute_egal_worst(As,new_valuations)
    c_res=compute_egal_cvar(As,valuations,p)
    print("worst opt",compute_egal_worst(As,new_valuations))
    print("cvar opt",compute_egal_cvar(As,valuations,p))
    print("**********Egalitarian***********")
    print("P",p)
    res_avg_egal=compute_optimal_allocation_single(As,np.array([mv]),egal=True)
    #print(res_avg[0][1])
   
    print("avg assigment",As[res_avg[0][1][0]])
    print("worst assignment",As[w_res[1]])
    print("cvar assignment",As[c_res[1]])
    print("Train Avg perf", compute_egal_cvar([As[res_avg_egal[0][1][0]]],valuations,p))
    print("Train Worst perf", compute_egal_cvar([As[w_res[1]]],valuations,p))
    print("Train CVaR perf", compute_egal_cvar([As[c_res[1]]],valuations,p))
    print("##################")

    print("Test Avg perf", compute_egal_cvar([As[res_avg_egal[0][1][0]]],valuations1,p))

    print("Test Worst perf", compute_egal_cvar([As[w_res[1]]],valuations1,p))
    print("Test CVaR perf", compute_egal_cvar([As[c_res[1]]],valuations1,p))
    print("**********")
    
        

In [81]:
# pvals=np.arange(0.02,0.25,0.02)
pvals=[0.04]
for pdx in pvals:
    train(pdx)
    

(40000, 2, 4)
(2, 4)
mean shape (2, 4)
std shape (2, 4)
0.04
0.005
8
std dev transpose [[0.01 0.01]
 [0.04 0.04]
 [0.05 0.05]
 [0.09 0.09]]
delta 0.005
alpha 0.005
lower bound [[0.35976659 0.36906634 0.35883293 0.25789927]
 [0.48976659 0.38906634 0.37883293 0.26789927]]
upper bound [[0.39146261 0.49585044 0.51731305 0.5431635 ]
 [0.52146261 0.51585044 0.53731305 0.5531635 ]]
selected points 33066
mean shape (2, 4)
std shape (2, 4)
0.04
0.005
8
std dev transpose [[0.01 0.01]
 [0.04 0.04]
 [0.05 0.05]
 [0.09 0.09]]
delta 0.005
alpha 0.005
lower bound [[0.35976659 0.36906634 0.35883293 0.25789927]
 [0.48976659 0.38906634 0.37883293 0.26789927]]
upper bound [[0.39146261 0.49585044 0.51731305 0.5431635 ]
 [0.52146261 0.51585044 0.53731305 0.5531635 ]]
selected points 33126
worst opt (0.8613259361321361, 3)
cvar opt (0.981945960201577, 6)
**********
P 0.04
avg assigment [[0. 0. 0. 1.]
 [0. 0. 1. 0.]]
worst assignment [[0. 1. 0. 0.]
 [1. 0. 0. 0.]]
cvar assignment [[0. 0. 1. 0.]
 [1. 0. 0. 0.

In [82]:
# pvals=np.arange(0.02,0.25,0.02)
pvals=[0.04]
for pdx in pvals:
    train_egal(pdx)

(40000, 2, 4)
(2, 4)
mean shape (2, 4)
std shape (2, 4)
0.04
0.005
8
std dev transpose [[0.01 0.01]
 [0.04 0.04]
 [0.05 0.05]
 [0.09 0.09]]
delta 0.005
alpha 0.005
lower bound [[0.35976659 0.36906634 0.35883293 0.25789927]
 [0.48976659 0.38906634 0.37883293 0.26789927]]
upper bound [[0.39146261 0.49585044 0.51731305 0.5431635 ]
 [0.52146261 0.51585044 0.53731305 0.5531635 ]]
selected points 33066
mean shape (2, 4)
std shape (2, 4)
0.04
0.005
8
std dev transpose [[0.01 0.01]
 [0.04 0.04]
 [0.05 0.05]
 [0.09 0.09]]
delta 0.005
alpha 0.005
lower bound [[0.35976659 0.36906634 0.35883293 0.25789927]
 [0.48976659 0.38906634 0.37883293 0.26789927]]
upper bound [[0.39146261 0.49585044 0.51731305 0.5431635 ]
 [0.52146261 0.51585044 0.53731305 0.5531635 ]]
selected points 33126
0.46936978357275577
worst opt (0.36907065317266335, 3)
0.46936978357275577
cvar opt (0.46936978357275577, 6)
**********Egalitarian***********
P 0.04
avg assigment [[0. 0. 0. 1.]
 [0. 0. 1. 0.]]
worst assignment [[0. 1. 0.